# 주제 : 서울시 구별 범죄 현황 분석

## 부제

+ CCTV와 범죄, 검거율에 관계가 있는가?
+ 강남 3구는 과연 범죄 발생률이 낮고 검거율이 높은가?
+ 우리동네는 어떤가?

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

## 1. Data Prepare

+ https://data.go.kr
+ 구글에서 "서울시 관서별 5대범죄 발생 검거 현황" 검색

In [5]:
crime_anal_police = pd.read_csv("data/seoul/crime_in_seoul.csv", 
                                    encoding="euc-kr", thousands=",")
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


In [6]:
crime_anal_police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관서명     31 non-null     object
 1   살인 발생   31 non-null     int64 
 2   살인 검거   31 non-null     int64 
 3   강도 발생   31 non-null     int64 
 4   강도 검거   31 non-null     int64 
 5   강간 발생   31 non-null     int64 
 6   강간 검거   31 non-null     int64 
 7   절도 발생   31 non-null     int64 
 8   절도 검거   31 non-null     int64 
 9   폭력 발생   31 non-null     int64 
 10  폭력 검거   31 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 2.8+ KB


## 2. EDA & Preprocessing

### 2.1. 관서명을 구 이름으로 변경

+ https://ko.wikipedia.org/wiki/서울특별시지방경찰청

+ kakao API를 이용하여 주소 획득
    - https://developers.kakao.com/
    - 키 : be9c427709d98b78ab9a851e8fb2ba4a
    - https://kauth.kakao.com/oauth/authorize?client_id=be9c427709d98b78ab9a851e8fb2ba4a&redirect_uri=https://localhost:8888&response_type=code


In [7]:
import requests

header = {"Authorization":"KakaoAK be9c427709d98b78ab9a851e8fb2ba4a"}
url = "https://dapi.kakao.com/v2/local/search/keyword.json?query='중부경찰서'"

requests.get(url, headers=header).json()

{'documents': [{'address_name': '서울 중구 저동2가 62-1',
   'category_group_code': 'PO3',
   'category_group_name': '공공기관',
   'category_name': '사회,공공기관 > 행정기관 > 경찰서',
   'distance': '',
   'id': '11151738',
   'phone': '182',
   'place_name': '서울중부경찰서',
   'place_url': 'http://place.map.kakao.com/11151738',
   'road_address_name': '서울 중구 수표로 27',
   'x': '126.98958691395208',
   'y': '37.56361812722828'},
  {'address_name': '경기 수원시 장안구 정자동 873-4',
   'category_group_code': 'PO3',
   'category_group_name': '공공기관',
   'category_name': '사회,공공기관 > 행정기관 > 경찰서',
   'distance': '',
   'id': '16176863',
   'phone': '182',
   'place_name': '수원중부경찰서',
   'place_url': 'http://place.map.kakao.com/16176863',
   'road_address_name': '경기 수원시 장안구 정자천로 199',
   'x': '126.99627805098389',
   'y': '37.29789068092514'},
  {'address_name': '부산 중구 대창동1가 40',
   'category_group_code': 'PO3',
   'category_group_name': '공공기관',
   'category_name': '사회,공공기관 > 행정기관 > 경찰서',
   'distance': '',
   'id': '10465693',
   'p

In [11]:
##### 관서명을 서울***결찰서로 변경

station_name = []

for name in crime_anal_police["관서명"]:
    station_name.append("서울" + name[:-1] +"경찰서")
    
station_name

['서울중부경찰서',
 '서울종로경찰서',
 '서울남대문경찰서',
 '서울서대문경찰서',
 '서울혜화경찰서',
 '서울용산경찰서',
 '서울성북경찰서',
 '서울동대문경찰서',
 '서울마포경찰서',
 '서울영등포경찰서',
 '서울성동경찰서',
 '서울동작경찰서',
 '서울광진경찰서',
 '서울서부경찰서',
 '서울강북경찰서',
 '서울금천경찰서',
 '서울중랑경찰서',
 '서울강남경찰서',
 '서울관악경찰서',
 '서울강서경찰서',
 '서울강동경찰서',
 '서울종암경찰서',
 '서울구로경찰서',
 '서울서초경찰서',
 '서울양천경찰서',
 '서울송파경찰서',
 '서울노원경찰서',
 '서울방배경찰서',
 '서울은평경찰서',
 '서울도봉경찰서',
 '서울수서경찰서']

In [19]:
station_address = []
station_lat = []    # 위도
station_lng = []    # 경도

for name in station_name:
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}".format(name)
    place = requests.get(url, headers=header).json()["documents"]
    
    station_address.append(place[0].get("address_name"))
    station_lat.append(place[0].get("y"))
    station_lng.append(place[0].get("x"))
    
    print(name + "---->" + place[0].get("address_name"))


서울중부경찰서---->서울 중구 저동2가 62-1
서울종로경찰서---->서울 종로구 경운동 90-18
서울남대문경찰서---->서울 중구 남대문로5가 567
서울서대문경찰서---->서울 서대문구 미근동 165
서울혜화경찰서---->서울 종로구 인의동 48-57
서울용산경찰서---->서울 용산구 원효로1가 12-12
서울성북경찰서---->서울 성북구 삼선동5가 301
서울동대문경찰서---->서울 동대문구 청량리동 229
서울마포경찰서---->서울 마포구 아현동 618-1
서울영등포경찰서---->서울 영등포구 당산동3가 2-11
서울성동경찰서---->서울 성동구 행당동 192-8
서울동작경찰서---->서울 동작구 노량진동 72-35
서울광진경찰서---->서울 광진구 구의동 254-32
서울서부경찰서---->서울 은평구 녹번동 177-15
서울강북경찰서---->서울 강북구 번동 415-15
서울금천경찰서---->서울 금천구 시흥동 1030
서울중랑경찰서---->서울 중랑구 신내동 810
서울강남경찰서---->서울 강남구 대치동 998
서울관악경찰서---->서울 관악구 봉천동 1695-5
서울강서경찰서---->서울 강서구 화곡동 980-27
서울강동경찰서---->서울 강동구 성내동 541-1
서울종암경찰서---->서울 성북구 하월곡동 27-5
서울구로경찰서---->서울 구로구 구로동 436
서울서초경찰서---->서울 서초구 서초동 1726-1
서울양천경찰서---->서울 양천구 신정동 321
서울송파경찰서---->서울 송파구 가락동 9
서울노원경찰서---->서울 노원구 하계동 250
서울방배경찰서---->서울 서초구 방배동 455-10
서울은평경찰서---->서울 은평구 불광동 산 24
서울도봉경찰서---->서울 도봉구 창동 17
서울수서경찰서---->서울 강남구 개포동 14


In [33]:
##### 받은 주소에서 구 이름만 추출

df = pd.DataFrame([station_address, station_lat, station_lng])
df = df.T
df.columns = ["구별", "위도", "경도"]
df["구별"] = df["구별"].str.split(" ").str[1]
df

,구별,위도,경도
0,중구,37.56361812722828,126.98958691395208
1,종로구,37.57553779077874,126.98489861889408
2,중구,37.55478043476484,126.97349729923079
3,서대문구,37.5649023404956,126.966790740735
4,종로구,37.5718915247338,126.998935936567
5,용산구,37.54117933635974,126.96777437089538
6,성북구,37.58989767143614,127.01625465308534
7,동대문구,37.5850804205685,127.045762797538
8,마포구,37.5510118582052,126.95394117396
9,영등포구,37.52585220719,126.90094457988


+ Google API 이용
    - https://cloud.google.com/
    - https://console.cloud.google.com/
    - pip install googlemaps